# Post-procesamiento

Este notebook describe los métodos de post-procesamiento que fueron aplicados sobre los resultados de las predicciones del modelo de U-Net.

## Poligonizado

La siguiente función aplica una rutina de poligonización sobre los resultados de la predicción del modelo y genera un archivo vectorial en formato GeoPackage (GPKG). La rutina aplica `gdal_polygonize.py` a cada chip resultado generando un GPKG para cada chip, y luego une todos estos archivos en uno solo, de manera eficiente.

Antes de unirlos también aplica un umbral sobre los valores de los rásteres, que en este caso representan la probabilidad (valores entre 0 y 1).

In [2]:
!pip install  /home/ro/satproc/ -e ./../

Processing /home/ro/satproc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Obtaining file:///home/ro/basurales


  Created wheel for pysatproc: filename=pysatproc-0.1.5.post1.dev1+gd0180c8-py2.py3-none-any.whl size=32494 sha256=9ed6cb0eefefed7405e8e2ad5219239ecca4c65d691b217bbb51832d7d99208a
  Stored in directory: /tmp/pip-ephem-wheel-cache-pc3nco96/wheels/6a/aa/bf/885699ad1244110053be14fc31fb2b566976c5a4b15c6d70a5
Successfully built pysatproc
  Attempting uninstall: pysatproc
    Found existing installation: pysatproc 0.1.4.post1.dev8+g0b2f480
    Uninstalling pysatproc-0.1.4.post1.dev8+g0b2f480:
      Successfully uninstalled pysatproc-0.1.4.post1.dev8+g0b2f480
  Attempting uninstall: basurales
    Found existing installation: basurales 0.0.post0.dev4+g62a921a.dirty
    Uninstalling basurales-0.0.post0.dev4+g62a921a.dirty:
      Successfully uninstalled basurales-0.0.post0.dev4+g62a921a.dirty
  Running setup.py develop for basurales


In [14]:
from satproc.postprocess.polygonize import polygonize 
from satproc.filter import filter_by_max_prob

In [4]:
!mkdir /home/ro/basurales/data/data_results/filtered_up05/

mkdir: cannot create directory ‘/home/ro/basurales/data/data_results/filtered_up05/’: File exists


In [5]:
filter_by_max_prob(input_dir='/home/ro/basurales/data/data_results/6D/100_100',
                   output_dir='/home/ro/basurales/data/data_results/filtered_up05/',
                   threshold=0.5)

100%|██████████| 178235/178235 [08:55<00:00, 332.61it/s]


In [15]:
input_path = '/home/ro/basurales/data/data_results/filtered_up05/'
output_path = '/home/ro/basurales/data/data_results/basurales_100x100_thr05.gpkg'


polygonize(threshold=0.5,
           input_dir=input_path,
           output=output_path)

100%|██████████| 25/25 [02:40<00:00,  6.41s/it]


## Filtrar por área mínima

Como último paso, filtramos aquellos polígonos de area inferior a 300 m², dado que consideramos que son falsos positivos. Para esto utilizamos `ogr2ogr` y una consulta SQL. El resultado se guarda en `data/results/predict_300.gpkg`

In [19]:
min_area = 35000
input_path = "/home/ro/basurales/data/data_results/basurales_100x100_thr05_utm.gpkg"
output_path = "/home/ro/basurales/data/data_results/basurales_100x100_thr05_up35000_utm.gpkg"

In [22]:
!ogrinfo -al $input_path | head -n25

INFO: Open of `/home/ro/basurales/data/data_results/basurales_100x100_thr05_utm.gpkg'
      using driver `GPKG' successful.

Layer name: basurales_100x100_thr05_utm
Geometry: Polygon
Feature Count: 3996255
Extent: (456388.761164, 5473248.338332) - (1072792.602149, 6316984.743222)
Layer SRS WKT:
PROJCRS["WGS 84 / UTM zone 20S",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 20S",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-63,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],


In [23]:
!ogr2ogr \
    -t_srs EPSG:32720 \
    -f "GPKG" \
    -sql "SELECT * FROM basurales_100x100_thr05_utm m WHERE ST_Area(geom) > $min_area" \
    -dialect SQLITE \
    -nln results \
    $output_path \
    $input_path